In [9]:
import os, openai
from openai import OpenAI
from langchain.chains import SequentialChain, LLMChain
from langchain.chains.base import Chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

from typing import Dict
from typing import Dict
from dotenv import load_dotenv

In [10]:
import sys
sys.path.append('../python')
from chatgpt_ocr import is_pdf_by_signature, encode_image, pdf_to_image, extract_json_from_string

In [13]:
# 환경 변수에서 API 키 로드
load_dotenv('../.env')
openai.api_key = os.getenv("CHATGPT-RECEIPT")
client = OpenAI(api_key=os.getenv("CHATGPT-RECEIPT"))

### OCR Chain 만들기

In [38]:
class OCRChain(Chain):
    """
    OpenAI를 이용해서 OCR을 진행하는 사용자 정의 체인.
    """
    def __init__(self):
        super().__init__()

    @property
    def input_keys(self):
        return ["image_path"]

    @property
    def output_keys(self):
        return ["ocr_response"]  # 출력값으로 Assistant의 응답을 반환

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        image_path = inputs["image_path"]
        # print(f"Image path: {image_path}")
        try:
            # 이미지 파일 열기
            if is_pdf_by_signature(image_path):
                base64_image = pdf_to_image(image_path)
            else:
                base64_image = encode_image(image_path)
            response = openai.chat.completions.create(
                model="gpt-4o",  # GPT-4 Vision 모델 사용
                messages=[
                    {
                        "role": "system", 
                        "content": "You are an assistant that extracts information from receipt images."
                    },
                    {
                        "role": "user", 
                        "content": [
                            {
                                "type": "text",
                                "text": '''다음 텍스트는 영수증의 정보입니다. 이 텍스트에서 가게 이름, 날짜, 항목, 총액을 분석하고, 아래의 JSON형식으로 결과를 반환해 주세요.
                                JSON 형식:
                                {
                                    "상호명": "가게 이름",
                                    "날짜": "YYYY-MM-DD",
                                    "항목": [
                                        {"이름": "상품1", "가격": 상품1 가격},
                                        {"이름": "상품2", "가격": 상품2 가격}
                                    ],
                                    "총액": 총액
                                }
                                반환할 JSON 형식은 반드시 위의 구조와 일치해야 하며, 불필요한 설명은 포함하지 마세요.
                                영수증 텍스트:
                                """
                                [여기에 영수증의 텍스트 또는 OCR로 추출한 내용이 들어갑니다]
                                """
                                결과:''' 
                            },{
                                "type": "image_url",
                                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                            }
                        ],
                    },
                ],
            )
            
            # 응답 파싱
            raw_response = extract_json_from_string(response.choices[0].message.content)
            # print(f'raw_response = {raw_response}')
            return {"ocr_response": raw_response}
        except Exception as e:
            
            return {"ocr_response": f"Error during OpenAI API call: {e}"}

### Assistant Chain 만들기

In [39]:
# Custom Chain 정의
class OpenAIAssistantChain(Chain):
    """
    OpenAI Assistant에 질의하는 사용자 정의 체인.
    """
    def __init__(self, prompt: PromptTemplate):
        super().__init__()
        self._prompt = prompt


    @property
    def input_keys(self):
        # PromptTemplate에서 정의된 입력 변수 사용
        return self._prompt.input_variables

    @property
    def output_keys(self):
        return ["assistant_response"]
    

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        formatted_prompt = self._prompt.format(**inputs)
        # OpenAI API 호출
        thread_id = 'thread_Twa2ruyrfpBGhOqESAnAYn6W'
        assistant_id = 'asst_7J3TrQfqCaD5dWXZpu7665l5'
        message = client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content= formatted_prompt
        )
        # Run 생성 및 실행
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant_id
        )
        # Run 완료 대기
        while run.status != "completed":
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        # 결과 메시지 가져오기
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        # print(f'messages = {messages}')
        # 결과 출력
        for i, message in enumerate(messages):
            if message.role == "assistant":
                if i == 0:
                    response = message.content[0].text.value
        # print(f'i = {i}, response = {response}')
                
        return {"assistant_response": response}
    

In [40]:
assistant_prompt = PromptTemplate(
    input_variables=["ocr_response"],
    template="""다음은 분석 지침입니다:
1. JSON 데이터에서 '상호명' 값을 확인하세요.
2. 상호명을 검색하여 업종을 추론해 보세요.
3. '상호명'을 기반으로 아래 카테고리 중 하나를 정확히 선택하세요:
    - 회의비
    - 교통비
    - 식대
4. 가능한 경우 단답형으로 항목만 답변하세요. (예: "식대")
5. 정확한 카테고리를 판단할 수 없는 경우:
    - "판단 필요"라고 답변하세요.
    - "판단 필요"인 이유를 1~2 문장으로 간단히 설명하세요.
6. 결과는 다음 형식으로 반환하세요:
    - 카테고리: [선택된 항목 또는 판단 필요]
    - 이유: [간단한 설명, 없을 경우 "없음"]

JSON 데이터:
{ocr_response}

답변:"""
)

# OpenAI Assistant 체인
assistant_chain = OpenAIAssistantChain(prompt=assistant_prompt)

# OCR 체인
ocr_chain = OCRChain()

In [41]:

image_path = "../image/IMG_9258.jpg"  # 실제 이미지 파일 경로를 여기에 설정

# SequentialChain 구성
sequential_chain = SequentialChain(
    chains=[ocr_chain, assistant_chain],
    input_variables=["image_path"],
    output_variables=["assistant_response"],
    verbose=True
)
    
# 실행
result = sequential_chain.invoke({"image_path": image_path})
print("\n최종 결과:")
print(result['assistant_response'])



> Entering new SequentialChain chain...

> Finished chain.

최종 결과:
카테고리: 판단 필요
이유: "현대프리미엄아울렛 김포점 바버"에서 구입한 '버버남성캐주얼'은 의류 구매를 나타내며, 제공된 카테고리(회의비, 교통비, 식대) 중 어디에도 명확하게 속하지 않습니다.


#### 최종 체인

In [ ]:
from langchain.chains import SequentialChain

# PromptTemplate 정의
assistant_prompt = PromptTemplate(
    input_variables=["ocr_response"],
    template="""다음은 분석 지침입니다:
1. JSON 데이터에서 '상호' 값을 확인하세요.
2. 상호를 검색해서 업종을 판단해 주세요.
3. '상호'를 기반으로 아래 카테고리 중 하나를 선택하세요:
    - 회의비
    - 교통비
    - 식대
4. 단답형으로 항목만 답변해 주세요. (ex, 출장비)
5. 정확한 항목이 없을 경우, "판단 필요"라고 답변해 주세요.
6. 판단이 필요한 경우에는 이유를 말해주세요.

JSON 데이터:
{ocr_response}
답변:"""
)

# OCRChain 초기화
ocr_chain = OCRChain()

# OpenAI Assistant 체인 초기화
assistant_chain = OpenAIAssistantChain(prompt=assistant_prompt)

# SequentialChain 구성
sequential_chain = SequentialChain(
    chains=[ocr_chain, assistant_chain],
    input_variables=["image_path"],
    output_variables=["assistant_response"],
    verbose=True
)

# 테스트 이미지 경로 설정
test_image_path = "../image/hyundai.jpg"

# 체인 실행
result = sequential_chain.invoke({"image_path": test_image_path})
print("\n최종 결과:")
print(result)